# Run notebooks

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
from glob import glob

import papermill as pm

import util

/glade/work/mclong/miniconda3/envs/_unison/lib/python3.7/site-packages/dask_jobqueue/core.py:20: FutureWarning: tmpfile is deprecated and will be removed in a future release. Please use dask.utils.tmpfile instead.
  from distributed.utils import tmpfile


## Get control structure

In [3]:
control = util.get_control_dict()
control

{'output_dir': './_output',
 'project': 'NCGD0011',
 'casename': 'smyle-fosi.regional.lat-range-basin.000',
 'notebooks': {'description': {'use_cluster': False,
   'parameter_groups': {'none': {'casename': 'smyle-fosi.regional.lat-range-basin.000'}},
   'kernel_name': '_unison'},
  'ocn-fgco2': {'use_cluster': True,
   'parameter_groups': {'none': {'casename': 'smyle-fosi.regional.lat-range-basin.000',
     'component': 'pop',
     'stream': 'h'}},
   'kernel_name': '_unison'},
  'ocn-tracer-views': {'use_cluster': True,
   'kernel_name': 'onets',
   'default_params': {'casename': 'smyle-fosi.regional.lat-range-basin.000',
    'component': 'pop',
    'stream': 'h'},
   'parameter_groups': {'temperature': {'variable_id': 'TEMP'},
    'salinity': {'variable_id': 'SALT'},
    'oxygen': {'variable_id': 'O2'}}}}}

## Spin up a cluster

The notebooks are configured to connect to this cluster.

In [4]:
cluster = util.get_Cluster(project=control['project'])
cluster.scale(32)
cluster

## Run all the notebooks

In [5]:
output_dir = control['output_dir']

files = glob(f"{output_dir}/*.ipynb")
for f in files:
    os.remove(f)

for nb, info in control['notebooks'].items():
    parameter_groups = info['parameter_groups']
    use_cluster = info['use_cluster']
    for key, parms in parameter_groups.items():

        input_path = f'{nb}.ipynb'
        output_path = (
            f'{output_dir}/{nb}-{key}.ipynb'
            if key != 'none' else f'{output_dir}/{nb}.ipynb'
        )

        # check notebook expectations
        nb_api = pm.inspect_notebook(input_path)

        if nb_api:
            parms_in = dict(**parms)
            if use_cluster:
                parms_in['cluster_scheduler_address'] = cluster.scheduler_address
        else:
            parms_in = {}

        print(f'executing {input_path}')
        o = pm.execute_notebook(
            input_path=input_path,
            output_path=output_path,
            kernel_name=info['kernel_name'],
            parameters=parms_in,
            engine_name='md_jinja',
            jinja_data=parms_in,
        )

executing description.ipynb


Executing:   0%|          | 0/1 [00:00<?, ?cell/s]

executing ocn-fgco2.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

executing ocn-tracer-views.ipynb


Executing:   0%|          | 0/15 [00:00<?, ?cell/s]

In [6]:
cluster.close()